# 1. Imagens com Deep Belief Networks

In [ ]:
import os
import sys
import timeit

import numpy

import theano
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams
from DBN import DBN , mnist_load_data

### a) Demonstrates how to train and test a Deep Belief Network.

This is demonstrated on MNIST

In [ ]:
finetune_lr=0.1
pretraining_epochs=100
pretrain_lr=0.1
k=1
training_epochs=200
dataset='Mnist.mat'
batch_size=100

datasets = mnist_load_data(dataset)
train_set_x, train_set_y = datasets[0]
test_set_x, test_set_y = datasets[1]

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size

In [ ]:
# numpy random generator
numpy_rng = numpy.random.RandomState(130)
print('... building the model')
# construct the Deep Belief Network
dbn = DBN(numpy_rng=numpy_rng, n_ins=28 * 28,
          hidden_layers_sizes=[1000,1000,1000],
          n_outs=10)

### b) Pretreinamento do modelo

print('... getting the pretraining functions')
pretraining_fns = dbn.pretraining_functions(train_set_x=train_set_x,
                                            batch_size=batch_size,
                                            k=k)

print('... pre-training the model')
start_time = timeit.default_timer()
# Pre-train layer-wise
```python
for i in range(dbn.n_layers):
    # go through pretraining epochs
    for epoch in range(pretraining_epochs):
        # go through the training set
        c = []
        for batch_index in range(n_train_batches):
            c.append(pretraining_fns[i](index=batch_index,
                                        lr=pretrain_lr))
        print('Pre-training layer %i, epoch %d, cost ' % (i, epoch), end=' ')
        print(numpy.mean(c, dtype='float64'))

end_time = timeit.default_timer()

print('The pretraining code, ' +  'ran for %.2fm' % ((end_time - start_time) / 60.), file=sys.stderr)
```
# end


In [ ]:
dbn.train(datasets = datasets , 
          batch_size = batch_size, 
          finetune_lr = finetune_lr, 
          n_train_batches = n_train_batches,
          training_epochs = training_epochs)